In [ ]:
import os

import pandas as pd

DateGet = '04-19-2023'
# File paths
rttm_file_path = '../AliceWhisperSync/StarFish_{}/diarization_output.rttm'.format(DateGet)
ast_file_path = '../AliceWhisperSync/StarFish_{}/'.format(DateGet)

# Function to calculate overlap duration
def calculate_overlap(ast_start, ast_end, rttm_start, rttm_end):
    return min(ast_end, rttm_end) - max(ast_start, rttm_start)

# Read the RTTM file
with open(rttm_file_path, 'r') as file:
    rttm_contents = file.readlines()

# Process the RTTM file
rttm_data = []
for line in rttm_contents:
    parts = line.split()
    speaker_class = parts[7]
    # Filter only specified classifications
    if speaker_class in ['CHI', 'FEM', 'KCHI', 'MAL']:
        file_name = parts[1]
        start_time = float(parts[3])
        duration = float(parts[4])
        end_time = start_time + duration
        rttm_data.append([file_name, start_time, end_time, speaker_class, duration])

# Convert to DataFrame for easier processing
rttm_df = pd.DataFrame(rttm_data, columns=['file_name', 'start_time', 'end_time', 'speaker_class', 'duration'])

for ast_file_get in os.listdir(ast_file_path):
    print(ast_file_get)
    if '_AST' in ast_file_get:
        print('Process: ', ast_file_get)
        # Read and process the AST file using pandas
        ast_df = pd.read_csv(os.path.join(ast_file_path, ast_file_get))
        ast_df['speaker_class'] = ''
        ast_file_name = '_'.join([ast_file_get.split('_')[0], ast_file_get.split('_')[1]])

        filtered_rttm_df = rttm_df[rttm_df['file_name'] == ast_file_name]

        # Iterate over each row in the AST dataframe
        for index, ast_row in ast_df.iterrows():
            ast_start = ast_row['start_sec']
            ast_end = ast_row['end_sec']

            # Find matching speaker segments from the RTTM dataframe
            matching_speakers = filtered_rttm_df[(filtered_rttm_df['start_time'] <= ast_end) & (filtered_rttm_df['end_time'] >= ast_start)]

            # Calculate the duration of overlap for each speaker classification
            if not matching_speakers.empty:
                matching_speakers['overlap_duration'] = matching_speakers.apply(
                    lambda row: calculate_overlap(ast_start, ast_end, row['start_time'], row['end_time']), axis=1)
                # Pick the speaker classification with the longest overlap duration
                longest_speaker_class = matching_speakers.loc[matching_speakers['overlap_duration'].idxmax(), 'speaker_class']
                ast_df.at[index, 'speaker_class'] = longest_speaker_class

        # Save the updated AST dataframe to a new CSV file
        updated_ast_file_path = '../AliceWhisperSync/StarFish_{}_SyncAW/Sync_{}_AW.csv'.format(DateGet, ast_file_name)

        ast_df['remove'] = False
        i = 1
        while i < len(ast_df):
            time_start = ast_df.iloc[i]['start_sec']
            j = i - 1
            while ast_df.iloc[j]['start_sec'] > time_start:
                #print(ast_df.iloc[j])
                ast_df.at[j, 'remove'] = True
                j = j - 1
            i += 1

        cleaned_df = ast_df[~ast_df['remove']].drop(columns=['remove'])
        cleaned_df.to_csv(updated_ast_file_path, index=False)

In [6]:
import pandas as pd
import os

# Read the StarFish file
starfish_file_path = '../AliceWhisperSync/StarFish_{}/StarFish_{}.csv'.format(DateGet, DateGet)
starfish_df = pd.read_csv(starfish_file_path)

# Function to adjust the Sync AW file based on the tone flag timestamp
def adjust_sync_file(sync_file_path, flag_time):
    # Read the Sync AW file
    sync_df = pd.read_csv(sync_file_path)

    # Adjust the start_sec and end_sec columns
    sync_df['start_sec'] = (sync_df['start_sec'] - flag_time).round(1)
    sync_df['end_sec'] = (sync_df['end_sec'] - flag_time).round(1)

    # Save the adjusted file
    adjusted_file_path = os.path.join('../AliceWhisperSync/StarFish_{}_SyncAW'.format(DateGet), os.path.basename(sync_file_path))
    sync_df.to_csv(adjusted_file_path, index=False)

    return adjusted_file_path

# Directory to save adjusted files
adjusted_dir = '../AliceWhisperSync/StarFish_{}_SyncAW'.format(DateGet)
os.makedirs(adjusted_dir, exist_ok=True)

# Process each Sync AW file based on the flag information from the StarFish file
for _, row in starfish_df.iterrows():
    file_name = row['file_name'].replace('.wav', '_AW.csv')  # Adjusting file name format
    sync_file_path = f'F:/AliceWhisperSync/StarFish_{DateGet}_SyncAW/Sync_{file_name}'
    flag_time = row['flag_start_time']

    # Adjust the file if it exists
    if os.path.exists(sync_file_path):
        adjusted_file_path = adjust_sync_file(sync_file_path, flag_time)
        print(f'File {file_name} adjusted and saved as {adjusted_file_path}')

File 064_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_064_01_AW.csv
File 067_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_067_01_AW.csv
File 062_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_062_01_AW.csv
File 074_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_074_01_AW.csv
File 075_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_075_01_AW.csv
File 070_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_070_01_AW.csv
File 073_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_073_01_AW.csv
File 065_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_065_01_AW.csv
File 071_01_AW.csv adjusted and saved as F:/AliceWhisperSync/StarFish_04-19-2023_SyncAW\Sync_071_01_AW.csv
File 063_01_AW.csv adjusted and saved

In [ ]:
import os

import pandas as pd

# Function to calculate overlap duration
def calculate_overlap(ast_start, ast_end, rttm_start, rttm_end):
    return min(ast_end, rttm_end) - max(ast_start, rttm_start)

def remove_last_part(input_string):
    # Find the position of the last underscore
    last_underscore_index = input_string.rfind('_')

    # Slice the string to keep everything before the last underscore
    return input_string[:last_underscore_index] if last_underscore_index != -1 else input_string

pth = 'F:/AliceWhisperSync/StarFish2223'
for item in os.listdir(pth):
    print(item)
    if 'ALICE' in item:
        DateGet = item.split('_')[1]
        ALICE_pth = os.path.join(pth, item)

        # File paths
        rttm_file_path = os.path.join(ALICE_pth, 'diarization_output.rttm')
        ast_file_path = os.path.join(pth, 'WhisperResults', remove_last_part(item))

        # Read the RTTM file
        with open(rttm_file_path, 'r') as file:
            rttm_contents = file.readlines()

        # Process the RTTM file
        rttm_data = []
        for line in rttm_contents:
            parts = line.split()
            speaker_class = parts[7]
            # Filter only specified classifications
            if speaker_class in ['CHI', 'FEM', 'KCHI', 'MAL']:
                file_name = parts[1]
                start_time = float(parts[3])
                duration = float(parts[4])
                end_time = start_time + duration
                rttm_data.append([file_name, start_time, end_time, speaker_class, duration])

        # Convert to DataFrame for easier processing
        rttm_df = pd.DataFrame(rttm_data, columns=['file_name', 'start_time', 'end_time', 'speaker_class', 'duration'])

        for ast_file_get in os.listdir(ast_file_path):
            print(ast_file_get)
            if '_AST' in ast_file_get:
                print('Process: ', ast_file_get)
                # Read and process the AST file using pandas
                ast_df = pd.read_csv(os.path.join(ast_file_path, ast_file_get))
                ast_df['speaker_class'] = ''
                ast_file_name = '_'.join([ast_file_get.split('_')[0], ast_file_get.split('_')[1]])

                filtered_rttm_df = rttm_df[rttm_df['file_name'] == ast_file_name]

                # Iterate over each row in the AST dataframe
                for index, ast_row in ast_df.iterrows():
                    ast_start = ast_row['start_sec']
                    ast_end = ast_row['end_sec']

                    # Find matching speaker segments from the RTTM dataframe
                    matching_speakers = filtered_rttm_df[(filtered_rttm_df['start_time'] <= ast_end) & (filtered_rttm_df['end_time'] >= ast_start)]

                    # Calculate the duration of overlap for each speaker classification
                    if not matching_speakers.empty:
                        matching_speakers['overlap_duration'] = matching_speakers.apply(
                            lambda row: calculate_overlap(ast_start, ast_end, row['start_time'], row['end_time']), axis=1)
                        # Pick the speaker classification with the longest overlap duration
                        longest_speaker_class = matching_speakers.loc[matching_speakers['overlap_duration'].idxmax(), 'speaker_class']
                        ast_df.at[index, 'speaker_class'] = longest_speaker_class

                # Save the updated AST dataframe to a new CSV file
                if not os.path.exists(os.path.join(pth, 'StarFish_{}_SyncAW'.format(DateGet))):
                    os.makedirs(os.path.join(pth, 'StarFish_{}_SyncAW'.format(DateGet)))
                updated_ast_file_path = os.path.join(pth, 'StarFish_{}_SyncAW/Sync_{}_AW.csv'.format(DateGet, ast_file_name))

                ast_df['remove'] = False
                i = 1
                while i < len(ast_df):
                    time_start = ast_df.iloc[i]['start_sec']
                    j = i - 1
                    while ast_df.iloc[j]['start_sec'] > time_start:
                        #print(ast_df.iloc[j])
                        ast_df.at[j, 'remove'] = True
                        j = j - 1
                    i += 1

                cleaned_df = ast_df[~ast_df['remove']].drop(columns=['remove'])
                cleaned_df.to_csv(updated_ast_file_path, index=False)

In [22]:
# Path to the directory containing the CSV files
pth = 'F:/AliceWhisperSync/StarFish2223'

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each file in the directory
for item in os.listdir(pth):
    if 'SyncAW' in item:
        print(item)
        # Parse the date from the file name
        DateGet = item.split('_')[1]

        # Construct the full path to the file
        file_path = os.path.join(pth, item)

        for AWfile in os.listdir(file_path):
            if 'Sync' in AWfile:

                print(AWfile)
                # Read the CSV file into a DataFrame
                df = pd.read_csv(os.path.join(file_path, AWfile))

                # Add 'recorder' column with the file name
                df['recorder'] = AWfile.split('.')[0]

                # Add 'date' column with the parsed date
                df['date'] = DateGet

                # Append the DataFrame to the list
                dataframes.append(df)

# Concatenate all DataFrames into one
aggregated_df = pd.concat(dataframes)

# Save the aggregated DataFrame to a new CSV file
aggregated_df.to_csv('F:/AliceWhisperSync/AggregatedData.csv', index=False)

StarFish_01-30-2023_SyncAW
Sync_061_01_AW.csv
Sync_062_01_AW.csv
Sync_063_01_AW.csv
Sync_064_01_AW.csv
Sync_065_01_AW.csv
Sync_066_01_AW.csv
Sync_067_01_AW.csv
Sync_070_01_AW.csv
Sync_071_01_AW.csv
Sync_072_01_AW.csv
Sync_073_01_AW.csv
Sync_074_01_AW.csv
Sync_075_01_AW.csv
StarFish_02-01-2023_SyncAW
Sync_061_01_AW.csv
Sync_062_01_AW.csv
Sync_063_01_AW.csv
Sync_064_01_AW.csv
Sync_065_01_AW.csv
Sync_066_01_AW.csv
Sync_067_01_AW.csv
Sync_069_01_AW.csv
Sync_070_01_AW.csv
Sync_071_01_AW.csv
Sync_072_01_AW.csv
Sync_073_01_AW.csv
Sync_074_01_AW.csv
Sync_075_01_AW.csv
StarFish_03-13-2023_SyncAW
Sync_062_01_AW.csv
Sync_063_01_AW.csv
Sync_064_01_AW.csv
Sync_065_01_AW.csv
Sync_066_01_AW.csv
Sync_067_01_AW.csv
Sync_069_01_AW.csv
Sync_070_01_AW.csv
Sync_071_01_AW.csv
Sync_074_01_AW.csv
Sync_075_01_AW.csv
StarFish_03-15-2023_SyncAW
Sync_062_01_AW.csv
Sync_063_01_AW.csv
Sync_064_01_AW.csv
Sync_065_01_AW.csv
Sync_066_01_AW.csv
Sync_067_01_AW.csv
Sync_068_01_AW.csv
Sync_069_01_AW.csv
Sync_070_01_AW.csv

In [4]:
import pandas as pd

all_df = pd.read_csv('F:/AliceWhisperSync/AggregatedData.csv')

all_df['to_remove'] = False

for i in range(2, len(all_df)):
    if all_df.loc[i, ['recorder', 'date', 'sentence']].equals(all_df.loc[i-1, ['recorder', 'date', 'sentence']]) and all_df.loc[i, ['recorder', 'date', 'sentence']].equals(all_df.loc[i-2, ['recorder', 'date', 'sentence']]):
        all_df.loc[i, 'to_remove'] = True

all_df = all_df[all_df['to_remove'] == False]

all_df.drop('to_remove', axis=1, inplace=True)

def adjust_end_sec(row):
    # Calculate the number of seconds to the next 2-minute boundary from start_sec
    next_boundary = (row['start_sec'] // 120 + 1) * 120
    # If end_sec crosses the next 2-minute boundary, adjust it
    if row['end_sec'] > next_boundary:
        return next_boundary
    else:
        return row['end_sec']

# Apply the function to each row
all_df['end_sec'] = all_df.apply(adjust_end_sec, axis=1)

all_df.to_csv('F:/AliceWhisperSync/Cleaned_AggregatedData.csv', index=False)

In [1]:
import pandas as pd
import os
import re

all_df = pd.read_csv('F:/AliceWhisperSync/Cleaned_AggregatedData.csv')
mapping_dict = {}
folder_path = 'F:/AliceWhisperSync/MAPPING_StarFish_2223'

for file in os.listdir(folder_path):
    if file.startswith('MAPPING_StarFish_2223_'):
        date = re.search('MAPPING_StarFish_2223_(.*).csv', file).group(1)
        mapping_df = pd.read_csv(os.path.join(folder_path, file))
        mapping_df.dropna(subset=[mapping_df.columns[9]], inplace=True)
        for index, row in mapping_df.iterrows():
            if row[mapping_df.columns[17]] == 'Child':
                recorder_number = int(row[mapping_df.columns[9]])
                starts_Ubi = str(row[mapping_df.columns[11]])
                expires_Ubi = str(row[mapping_df.columns[12]])
                speaker_type = row[mapping_df.columns[17]]
                mapping_dict[(date, recorder_number)] = (starts_Ubi, expires_Ubi, row[mapping_df.columns[17]])

def extract_recorder_number(recorder):
    match = re.search('Sync_(\d+)_(\d+)_AW', recorder)
    if match:
        return match.group(1)
    return None

def get_ubis_and_type(row):
    key = (str(row['date']), int(row['recorder_number']))
    if key in mapping_dict:
        starts_Ubi, expires_Ubi, speaker_type = mapping_dict[key]
        return pd.Series([starts_Ubi, expires_Ubi, speaker_type])
    return pd.Series([None, None, None])

all_df['recorder_number'] = all_df['recorder'].apply(extract_recorder_number)

# Apply the function and split the returned series into separate columns
all_df[['starts_Ubi', 'expires_Ubi', 'speaker_type']] = all_df.apply(get_ubis_and_type, axis=1)

# Now you can filter all_df based on 'speaker_type' being 'Child'
filtered_df = all_df[all_df['speaker_type'] == 'Child']

#filtered_df = all_df[all_df.apply(lambda row: mapping_dict.get((str(row['date']), int(row['recorder_number'])))[2] == 'Child', axis=1)]

filtered_df.drop('recorder_number', axis=1, inplace=True)
filtered_df

,Unnamed: 0,start_sec,end_sec,lang,language_t3,probability_t3,sentence,speaker_class,recorder,date,WC,starts_Ubi,expires_Ubi,speaker_type
0,0,0.0,15.7,en,"['en', 'es', 'haw']","{'en': 0.9358899593353271, 'es': 0.03832525387...","It's 8.34.12am. This is Lina1 on Sony 61, Chi...",FEM,Sync_061_01_AW,01-30-2023,12,8:55,12:49,Child
1,1,15.7,44.2,en,"['en', 'haw', 'es']","{'en': 0.9195829033851624, 'haw': 0.0515678636...",Debbie. I'm a whore. It's 8.34.46am. This is ...,FEM,Sync_061_01_AW,01-30-2023,14,8:55,12:49,Child
2,2,44.2,74.2,en,"['en', 'haw', 'es']","{'en': 0.9700366258621216, 'haw': 0.0153719000...","January 30, 2023. It's 8.35.16am. This is Son...",FEM,Sync_061_01_AW,01-30-2023,11,8:55,12:49,Child
3,3,74.2,103.2,en,"['en', 'es', 'cs']","{'en': 0.9178056716918945, 'es': 0.01618556864...","Debbie, January 30, 2023. It's 8.35.48am. Thi...",FEM,Sync_061_01_AW,01-30-2023,12,8:55,12:49,Child
4,4,103.2,117.7,en,"['en', 'haw', 'nn']","{'en': 0.9576286673545837, 'haw': 0.0080133983...","3. Subject 31, Lina 2, and yeah, right. Yeah....",FEM,Sync_061_01_AW,01-30-2023,11,8:55,12:49,Child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803785,4726,15773.2,15795.2,en,"['en', 'haw', 'nn']","{'en': 0.505650520324707, 'haw': 0.35071983933...",this is Lena 13 p.m. which is 0 3 4 4 5 5 and...,FEM,Sync_076_01_AW,12-07-2022,10,None,None,None
803786,4727,15795.2,15813.2,en,"['en', 'nl', 'de']","{'en': 0.9262458086013794, 'nl': 0.01371836382...","all these are about to die, these Lenas they ...",FEM,Sync_076_01_AW,12-07-2022,12,None,None,None
803787,4728,15813.2,15826.2,en,"['en', 'es', 'de']","{'en': 0.9783369302749634, 'es': 0.00806262064...","I mean these are old I think, they are a coup...",FEM,Sync_076_01_AW,12-07-2022,18,None,None,None
803788,4729,15826.4,15840.0,en,"['en', 'ja', 'es']","{'en': 0.9844958782196045, 'ja': 0.00466062314...",this was lab 2 using Lena lab 1 a.m. fry whic...,FEM,Sync_076_01_AW,12-07-2022,12,None,None,None


In [2]:
filtered_df['sentence'] = filtered_df['sentence'].apply(lambda x: ' '.join(str(x).split()))

filtered_df['duplicate_flag'] = filtered_df['sentence'].shift(1) == filtered_df['sentence']
filtered_df['duplicate_flag'] &= filtered_df['sentence'].shift(2) == filtered_df['sentence']

filtered_df = filtered_df[~filtered_df['duplicate_flag']].drop(columns=['duplicate_flag'])

In [3]:
filtered_df

,Unnamed: 0,start_sec,end_sec,lang,language_t3,probability_t3,sentence,speaker_class,recorder,date,WC,starts_Ubi,expires_Ubi,speaker_type
0,0,0.0,15.7,en,"['en', 'es', 'haw']","{'en': 0.9358899593353271, 'es': 0.03832525387...","It's 8.34.12am. This is Lina1 on Sony 61, Chil...",FEM,Sync_061_01_AW,01-30-2023,12,8:55,12:49,Child
1,1,15.7,44.2,en,"['en', 'haw', 'es']","{'en': 0.9195829033851624, 'haw': 0.0515678636...",Debbie. I'm a whore. It's 8.34.46am. This is L...,FEM,Sync_061_01_AW,01-30-2023,14,8:55,12:49,Child
2,2,44.2,74.2,en,"['en', 'haw', 'es']","{'en': 0.9700366258621216, 'haw': 0.0153719000...","January 30, 2023. It's 8.35.16am. This is Sony...",FEM,Sync_061_01_AW,01-30-2023,11,8:55,12:49,Child
3,3,74.2,103.2,en,"['en', 'es', 'cs']","{'en': 0.9178056716918945, 'es': 0.01618556864...","Debbie, January 30, 2023. It's 8.35.48am. This...",FEM,Sync_061_01_AW,01-30-2023,12,8:55,12:49,Child
4,4,103.2,117.7,en,"['en', 'haw', 'nn']","{'en': 0.9576286673545837, 'haw': 0.0080133983...","3. Subject 31, Lina 2, and yeah, right. Yeah. ...",FEM,Sync_061_01_AW,01-30-2023,11,8:55,12:49,Child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803785,4726,15773.2,15795.2,en,"['en', 'haw', 'nn']","{'en': 0.505650520324707, 'haw': 0.35071983933...",this is Lena 13 p.m. which is 0 3 4 4 5 5 and ...,FEM,Sync_076_01_AW,12-07-2022,10,None,None,None
803786,4727,15795.2,15813.2,en,"['en', 'nl', 'de']","{'en': 0.9262458086013794, 'nl': 0.01371836382...","all these are about to die, these Lenas they d...",FEM,Sync_076_01_AW,12-07-2022,12,None,None,None
803787,4728,15813.2,15826.2,en,"['en', 'es', 'de']","{'en': 0.9783369302749634, 'es': 0.00806262064...","I mean these are old I think, they are a coupl...",FEM,Sync_076_01_AW,12-07-2022,18,None,None,None
803788,4729,15826.4,15840.0,en,"['en', 'ja', 'es']","{'en': 0.9844958782196045, 'ja': 0.00466062314...",this was lab 2 using Lena lab 1 a.m. fry which...,FEM,Sync_076_01_AW,12-07-2022,12,None,None,None


In [4]:
filtered_df.to_csv('F:/AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

In [18]:
import re
import pandas as pd

def clean_transcription(transcription):
    return re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', transcription)

def word_count(s):
    try:
        # Remove all non-character symbols
        cleaned_str = clean_transcription(s).strip()
        return len(cleaned_str.split())
    except:
        return 0

def replace_speaker_class(speaker_class):
    if speaker_class in ['MAL', 'FEM']:
        return 'Teacher'
    elif speaker_class in ['CHI', 'KCHI']:
        return 'Child'
    else:
        return speaker_class

all_df = pd.read_csv('F:/AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

all_df['WC'] = all_df['sentence'].astype(str).apply(word_count)

all_df['speaker_class'] = all_df['speaker_class'].apply(replace_speaker_class)


# Calculate average WC for different speaker_class in different date
average_wc_per_class_and_date = all_df.groupby(['speaker_class', 'date'])['WC'].mean().reset_index()

# Calculate overall average WC for different speaker_class
overall_average_wc_per_class = all_df.groupby('speaker_class')['WC'].mean().reset_index()

# Display the results
print("Average WC per speaker class and date:")
print(average_wc_per_class_and_date)

print("\nOverall average WC per speaker class:")
print(overall_average_wc_per_class)

average_wc_per_class_and_date.to_csv('F:/AliceWhisperSync/average_wc_per_class_and_date.csv', index=False)
overall_average_wc_per_class.to_csv('F:/AliceWhisperSync/overall_average_wc_per_class.csv', index=False)

Average WC per speaker class and date:
   speaker_class        date        WC
0          Child  01-30-2023  3.092506
1          Child  02-01-2023  3.085437
2          Child  03-13-2023  3.652429
3          Child  03-15-2023  3.303292
4          Child  04-17-2023  3.400315
5          Child  04-19-2023  3.313509
6          Child  06-15-2023  3.478026
7          Child  07-11-2023  3.444464
8          Child  07-13-2023  3.294516
9          Child  09-09-2022  3.033669
10         Child  10-21-2022  3.219097
11         Child  11-14-2022  3.126064
12         Child  11-16-2022  3.301792
13         Child  12-05-2022  3.112747
14         Child  12-07-2022  3.267220
15       Teacher  01-30-2023  3.967947
16       Teacher  02-01-2023  3.914663
17       Teacher  03-13-2023  5.032072
18       Teacher  03-15-2023  4.352418
19       Teacher  04-17-2023  4.099497
20       Teacher  04-19-2023  4.094401
21       Teacher  06-15-2023  4.064802
22       Teacher  07-11-2023  4.341469
23       Teacher  07-13-2

In [19]:
# Overall Speaker Distribution
overall_speaker_distribution = all_df['speaker_class'].value_counts().reset_index()
overall_speaker_distribution.columns = ['speaker_class', 'overall_count']

# Speaker Distribution for Each Date
speaker_distribution_per_date = all_df.groupby('date')['speaker_class'].value_counts().unstack(fill_value=0)

# Display the results
print("Overall Speaker Distribution:")
print(overall_speaker_distribution)
print("\nSpeaker Distribution for Each Date:")
print(speaker_distribution_per_date)

overall_speaker_distribution.to_csv('F:/AliceWhisperSync/overall_speaker_distribution.csv', index=False)
speaker_distribution_per_date.to_csv('F:/AliceWhisperSync/speaker_distribution_per_date.csv', index=False)

Overall Speaker Distribution:
  speaker_class  overall_count
0       Teacher         311835
1         Child         286261

Speaker Distribution for Each Date:
speaker_class  Child  Teacher
date                         
01-30-2023     22377    23461
02-01-2023     24205    20425
03-13-2023     18425    18053
03-15-2023     22203    27439
04-17-2023     24101    26443
04-19-2023     22363    24756
06-15-2023     17043    20061
07-11-2023     17160    17536
07-13-2023     15663    17524
09-09-2022      2881     3273
10-21-2022     12798    19531
11-14-2022     19149    22374
11-16-2022     19699    22219
12-05-2022     25198    25721
12-07-2022     22996    23019


In [2]:
import re
import os
import pandas as pd

def clean_transcription(transcription):
    return re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ\s]', '', transcription)

def word_count(s):
    try:
        # Remove all non-character symbols
        cleaned_str = clean_transcription(s).strip()
        return len(cleaned_str.split())
    except:
        return 0

def replace_speaker_class(speaker_class):
    if speaker_class in ['MAL', 'FEM']:
        return 'Teacher'
    else:
        return speaker_class

all_df = pd.read_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

mapping_dict = {}
folder_path = '../AliceWhisperSync/MAPPING_StarFish_2223'

for file in os.listdir(folder_path):
    if file.startswith('MAPPING_StarFish_2223_'):
        date = re.search('MAPPING_StarFish_2223_(.*).csv', file).group(1)
        mapping_df = pd.read_csv(os.path.join(folder_path, file))
        mapping_df.dropna(subset=[mapping_df.columns[9]], inplace=True)
        for index, row in mapping_df.iterrows():
            if row[mapping_df.columns[17]] == 'Child':
                recorder_number = int(row[mapping_df.columns[9]])
                mapping_dict[(date, recorder_number)] = row[mapping_df.columns[2]]

def extract_recorder_number(recorder):
    match = re.search('Sync_(\d+)_(\d+)_AW', recorder)
    if match:
        return match.group(1)
    return None

all_df['recorder_number'] = all_df['recorder'].apply(extract_recorder_number)

all_df['subject ID'] = all_df.apply(lambda row: mapping_dict.get((str(row['date']), int(row['recorder_number']))), axis=1)

all_df['WC'] = all_df['sentence'].astype(str).apply(word_count)

all_df['speaker_class'] = all_df['speaker_class'].apply(replace_speaker_class)

all_df.to_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI_test.csv')

# Calculate average WC for different speaker_class in different date
average_wc_per_class_and_date = all_df.groupby(['speaker_class', 'date', 'subject ID'])['WC'].mean().reset_index()

# Calculate overall average WC for different speaker_class
overall_average_wc_per_class = all_df.groupby('subject ID')['WC'].mean().reset_index()

# Display the results
print("Average WC per speaker class and date:")
print(average_wc_per_class_and_date)

print("\nOverall average WC per speaker class:")
print(overall_average_wc_per_class)

average_wc_per_class_and_date.to_csv('../AliceWhisperSync/average_wc_per_class_and_date.csv', index=False)
overall_average_wc_per_class.to_csv('../AliceWhisperSync/overall_average_wc_per_class.csv', index=False)

# Overall Speaker Distribution
overall_speaker_distribution = all_df['speaker_class'].value_counts().reset_index()
overall_speaker_distribution.columns = ['speaker_class', 'overall_count']

# Speaker Distribution for Each Date
speaker_distribution_per_date = all_df.groupby('subject ID')['speaker_class'].value_counts().unstack(fill_value=0)

# Display the results
print("Overall Speaker Distribution:")
print(overall_speaker_distribution)
print("\nSpeaker Distribution for Each Date:")
print(speaker_distribution_per_date)

overall_speaker_distribution.to_csv('../AliceWhisperSync/overall_speaker_distribution.csv', index=False)
speaker_distribution_per_date.to_csv('../AliceWhisperSync/speaker_distribution_per_date.csv', index=False)

Average WC per speaker class and date:
    speaker_class        date           subject ID        WC
0             CHI  01-30-2023  DS_STARFISH_2223_27  2.959545
1             CHI  01-30-2023  DS_STARFISH_2223_28  3.068462
2             CHI  01-30-2023  DS_STARFISH_2223_29  2.969309
3             CHI  01-30-2023  DS_STARFISH_2223_30  3.353873
4             CHI  01-30-2023  DS_STARFISH_2223_31  2.893089
..            ...         ...                  ...       ...
448       Teacher  12-07-2022  DS_STARFISH_2223_42  4.077053
449       Teacher  12-07-2022  DS_STARFISH_2223_43  3.938250
450       Teacher  12-07-2022  DS_STARFISH_2223_44  3.982022
451       Teacher  12-07-2022  DS_STARFISH_2223_45  4.077237
452       Teacher  12-07-2022  DS_STARFISH_2223_46  4.000000

[453 rows x 4 columns]

Overall average WC per speaker class:
             subject ID        WC
0   DS_STARFISH_2223_27  3.558244
1   DS_STARFISH_2223_28  3.610181
2   DS_STARFISH_2223_29  3.636384
3   DS_STARFISH_2223_30  3.816

In [1]:
#CHIList = ['CHI', 'KCHI']
CHIList = ['KCHI']

In [2]:
import pandas as pd
from tqdm import tqdm

all_df = pd.read_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

all_responses_df = pd.DataFrame()
summary_list = []
processed_dfs = []

response_time_window = 2.5
grouped = all_df.groupby(['subject ID', 'date'])
total_groups = len(grouped)

for i, ((recorder, date), group_df) in enumerate(tqdm(grouped, total=total_groups, desc="Processing")):
    response_df = pd.DataFrame(columns=['Question', 'Response', 'Question Time Start', 'Question Time End', 'Response Time Start', 'Response Time End', 'Latencies', 'Question Speaker', 'Response Speaker', 'File Name'])
    total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs = 0, 0, 0, 0, 0

    num_rows = len(group_df) - 1

    # 遍历当前组的每一行
    index = -1
    for _, row in group_df.iterrows():
        index += 1
        if index == num_rows:
            break
        speaker = row['speaker_class']  # 假设speaker_class是Column O
        sentence = row['sentence']  # 假设sentence是Column E
        question_start = row['start_sec']
        question_end = row['end_sec']
        file_name = recorder + '_' + date

        total_Sen += 1

        if speaker in ['FEM'] and '?' in str(sentence):
            Q_sen += 1
            index_2 = index + 1
            response_row = group_df.iloc[index_2]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])

                if response_speaker in ['FEM'] and '?' in str(response_sen):
                    break

                if response_speaker in CHIList and response_sen != 'nan':
                    Q_Rs += 1
                    new_row = pd.DataFrame({
                        'Question': [sentence],
                        'Response': [response_sen],
                        'Question Time Start': [question_start],
                        'Question Time End': [question_end],
                        'Response Time Start': [response_row['start_sec']],
                        'Response Time End': [response_row['end_sec']],
                        'Latencies': [response_row['start_sec'] - question_end],
                        'Question Speaker': [speaker],
                        'Response Speaker': [response_speaker],
                        'File Name': [file_name]
                    })
                    
                    response_df = pd.concat([response_df, new_row], ignore_index=True)

                    group_df.at[index_2, 'Resp'] = 1
                    break

                index_2 += 1
                if index_2 > num_rows:
                    break
                response_row = group_df.iloc[index_2]

        elif speaker in ['FEM']:
            NQ_sen += 1
            index_3 = index + 1
            response_row = group_df.iloc[index_3]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])

                if response_speaker in CHIList:
                    NQ_Rs += 1
                    group_df.at[index_3, 'Resp'] = 1
                    break
                index_3 += 1
                if index_3 > num_rows:
                    break
                response_row = group_df.iloc[index_3]

    summary_list.append((recorder, date, total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs, file_name))
    all_responses_df = pd.concat([all_responses_df, response_df])
    processed_dfs.append(group_df[group_df['speaker_class'].isin(CHIList)])

# 将summary_list转换为DataFrame以更好地查看和分析
summary_df = pd.DataFrame(summary_list, columns=['Recorder', 'Date', 'Total Sentences', 'Question Sentences', 'Non-Question Sentences', 'Question Responses', 'Non-Question Responses', 'File Name'])

# 输出结果
# all_responses_df, summary_df, processed_dfs

Processing:   0%|          | 0/151 [00:00<?, ?it/s]/var/folders/ld/7cclcyj90kj1h2swhyy596s80000gp/T/ipykernel_47728/3573986033.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  response_df = pd.concat([response_df, new_row], ignore_index=True)
Processing:   1%|          | 1/151 [00:00<01:00,  2.46it/s]/var/folders/ld/7cclcyj90kj1h2swhyy596s80000gp/T/ipykernel_47728/3573986033.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  response_df = pd.concat([response_df, new_row], ignore_index=True

In [23]:
all_responses_df

,Question,Response,Question Time Start,Question Time End,Response Time Start,Response Time End,Latencies,Question Speaker,Response Speaker,File Name
0,Can you go say it?,Wee,1180.8,1181.8,1182.8,1184.8,1.0,FEM,KCHI,DS_STARFISH_2223_27_1/30/23
1,Who is going to be four?,Ignacio.,1214.0,1217.0,1217.0,1219.0,0.0,FEM,KCHI,DS_STARFISH_2223_27_1/30/23
2,"One and two, what number is that?",Twelve.,1336.0,1338.0,1338.0,1340.0,0.0,FEM,KCHI,DS_STARFISH_2223_27_1/30/23
3,"Luna, ready?",We're going to go outside.,1479.0,1480.0,1481.0,1482.0,1.0,FEM,KCHI,DS_STARFISH_2223_27_1/30/23
4,In a chair?,Then sit next to me.,2194.0,2196.0,2196.0,2198.0,0.0,FEM,KCHI,DS_STARFISH_2223_27_1/30/23
...,...,...,...,...,...,...,...,...,...,...
68,"Right, you're counting?","Oh, she has four.",13621.0,13623.0,13623.0,13625.0,0.0,FEM,KCHI,DS_STARFISH_2223_46_6/15/23
69,There's nothing else?,Jelly,13788.0,13790.0,13790.0,13792.0,0.0,FEM,KCHI,DS_STARFISH_2223_46_6/15/23
70,"Thank you, you wanna see?",That's me.,13828.0,13830.0,13830.0,13832.0,0.0,FEM,KCHI,DS_STARFISH_2223_46_6/15/23
71,Is it dirty?,I like it better than today,13930.0,13932.0,13934.0,13936.0,2.0,FEM,KCHI,DS_STARFISH_2223_46_6/15/23


In [3]:
summary_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_TeacherQ.xlsx')
all_responses_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Details_KCHI_FEM_TeacherQ.xlsx')

In [4]:
import pandas as pd
from tqdm import tqdm

all_df = pd.read_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

all_responses_df = pd.DataFrame()
summary_list = []
processed_dfs = []

response_time_window = 2.5
grouped = all_df.groupby(['subject ID', 'date'])
total_groups = len(grouped)

for i, ((recorder, date), group_df) in enumerate(tqdm(grouped, total=total_groups, desc="Processing")):
    response_df = pd.DataFrame(columns=['Question', 'Response', 'Question Time Start', 'Question Time End', 'Response Time Start', 'Response Time End', 'Latencies', 'Question Speaker', 'Response Speaker', 'File Name'])
    total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs = 0, 0, 0, 0, 0

    num_rows = len(group_df) - 1

    # 遍历当前组的每一行
    index = -1
    for _, row in group_df.iterrows():
        index += 1
        if index == num_rows:
            break
        speaker = row['speaker_class']  # 假设speaker_class是Column O
        sentence = row['sentence']  # 假设sentence是Column E
        question_start = row['start_sec']
        question_end = row['end_sec']
        file_name = recorder + '_' + date

        total_Sen += 1

        if speaker in CHIList and '?' in str(sentence):
            Q_sen += 1
            index_2 = index + 1
            response_row = group_df.iloc[index_2]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])

                if response_speaker in CHIList and '?' in str(response_sen):
                    break

                if response_speaker in ['FEM'] and response_sen != 'nan':
                    Q_Rs += 1
                    new_row = pd.DataFrame({
                        'Question': [sentence],
                        'Response': [response_sen],
                        'Question Time Start': [question_start],
                        'Question Time End': [question_end],
                        'Response Time Start': [response_row['start_sec']],
                        'Response Time End': [response_row['end_sec']],
                        'Latencies': [response_row['start_sec'] - question_end],
                        'Question Speaker': [speaker],
                        'Response Speaker': [response_speaker],
                        'File Name': [file_name]
                    })
                    
                    response_df = pd.concat([response_df, new_row], ignore_index=True)
                    group_df.at[index_2, 'Resp'] = 1
                    break

                index_2 += 1
                if index_2 > num_rows:
                    break
                response_row = group_df.iloc[index_2]

        elif speaker in CHIList:
            NQ_sen += 1
            index_3 = index + 1
            response_row = group_df.iloc[index_3]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])

                if response_speaker in ['FEM']:
                    NQ_Rs += 1
                    group_df.at[index_3, 'Resp'] = 1
                    break
                index_3 += 1
                if index_3 > num_rows:
                    break
                response_row = group_df.iloc[index_3]

    summary_list.append((recorder, date, total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs, file_name))
    all_responses_df = pd.concat([all_responses_df, response_df])
    processed_dfs.append(group_df[group_df['speaker_class'].isin(['FEM'])])

# 将summary_list转换为DataFrame以更好地查看和分析
summary_df = pd.DataFrame(summary_list, columns=['Recorder', 'Date', 'Total Sentences', 'Question Sentences', 'Non-Question Sentences', 'Question Responses', 'Non-Question Responses', 'File Name'])

Processing:   0%|          | 0/151 [00:00<?, ?it/s]/var/folders/ld/7cclcyj90kj1h2swhyy596s80000gp/T/ipykernel_47728/1109077885.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  response_df = pd.concat([response_df, new_row], ignore_index=True)
Processing:   1%|          | 1/151 [00:00<00:41,  3.60it/s]/var/folders/ld/7cclcyj90kj1h2swhyy596s80000gp/T/ipykernel_47728/1109077885.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  response_df = pd.concat([response_df, new_row], ignore_index=True

In [26]:
all_responses_df

,Question,Response,Question Time Start,Question Time End,Response Time Start,Response Time End,Latencies,Question Speaker,Response Speaker,File Name
0,How are you?,Good.,1472.0,1473.0,1475.0,1476.0,2.0,KCHI,FEM,Sync_061_01_AW_01-30-2023
1,Do you see the guana?,One became...,2823.0,2825.0,2825.0,2827.0,0.0,KCHI,FEM,Sync_061_01_AW_01-30-2023
2,Do you want to dry her hands?,What would you like? You'd like a choo-choo too?,3884.0,3886.0,3888.0,3890.0,2.0,KCHI,FEM,Sync_061_01_AW_01-30-2023
3,Can she do it?,"Alex, put on your mask.",3920.0,3922.0,3922.0,3924.0,0.0,KCHI,FEM,Sync_061_01_AW_01-30-2023
4,Can I help you?,I'm here.,4168.0,4170.0,4170.0,4172.0,0.0,KCHI,FEM,Sync_061_01_AW_01-30-2023
...,...,...,...,...,...,...,...,...,...,...
16,What?,What did you do?,6950.0,6952.0,6952.0,6954.0,0.0,KCHI,FEM,Sync_083_01_AW_07-13-2023
17,No one's stars?,Yeah.,7207.0,7208.0,7208.0,7209.0,0.0,KCHI,FEM,Sync_083_01_AW_07-13-2023
18,Stars?,Yep.,7213.0,7214.0,7214.0,7215.1,0.0,KCHI,FEM,Sync_083_01_AW_07-13-2023
19,What happened?,Bye.,9390.0,9392.0,9394.0,9396.0,2.0,KCHI,FEM,Sync_083_01_AW_07-13-2023


In [5]:
summary_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_KChildQ.xlsx')
all_responses_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Details_KCHI_FEM_KChildQ.xlsx')

In [30]:
import pandas as pd
from tqdm import tqdm
import re

def word_count_unq(s):
    try:
        # Remove all non-character symbols and convert to lowercase
        cleaned_str = clean_transcription(s).strip()
        # Use a set to keep track of unique words
        unique_words = set(cleaned_str.split())
        return len(unique_words)
    except Exception as e:
        print(f"Error: {e}")
        return 0

def word_count(s):
    try:
        # Remove all non-character symbols
        cleaned_str = clean_transcription(s).strip()
        return len(cleaned_str.split())
    except:
        return 0

def clean_transcription(transcription):
    return re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚüÜ0-9\s]', '', transcription)

all_df = pd.read_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')
#all_df['LexDiv'] = all_df['sentence'].apply(word_count_unq)
#all_df['WordCount'] = all_df['sentence'].apply(word_count)

grouped = all_df.groupby(['subject ID', 'date', 'speaker_class'])
total_groups = len(grouped)

All_ratio = []

'''
for i, ((recorder, date, speaker_class), group_df) in enumerate(tqdm(grouped, total=total_groups, desc="Processing")):
    num_rows = 0
    ratio_sum = 0
    for _, row in group_df.iterrows():
        if row['WordCount'] > 0:
            num_rows += 1
            ratio_sum += float(int(row['LexDiv']) / int(row['WordCount']))
    ratio_avg = ratio_sum / num_rows
    All_ratio.append([recorder, date, speaker_class, ratio_avg])
'''

for i, ((recorder, date, speaker_class), group_df) in enumerate(tqdm(grouped, total=total_groups, desc="Processing")):
    num_rows = 0
    ratio_sum = 0
    content_all = []
    for _, row in group_df.iterrows():
        content_all.append(str(row['sentence']))
    agg_content = ' '.join(content_all)
    ratio_avg = word_count_unq(agg_content) / word_count(agg_content)
    All_ratio.append([recorder, date, speaker_class, ratio_avg])

All_ratio_df = pd.DataFrame(All_ratio, columns=['Subject ID', 'date', 'speaker_class', 'number of unique words divided by total words per utterance'])
All_ratio_df

Processing:  80%|████████  | 604/755 [00:11<00:02, 50.90it/s]


,Subject ID,date,speaker_class,number of unique words divided by total words per utterance
0,DS_STARFISH_2223_27,1/30/23,CHI,0.270904
1,DS_STARFISH_2223_27,1/30/23,FEM,0.159269
2,DS_STARFISH_2223_27,1/30/23,KCHI,0.284377
3,DS_STARFISH_2223_27,1/30/23,MAL,0.844444
4,DS_STARFISH_2223_27,10/21/22,CHI,0.332929
...,...,...,...,...
599,DS_STARFISH_2223_46,4/19/23,MAL,0.783784
600,DS_STARFISH_2223_46,6/15/23,CHI,0.300746
601,DS_STARFISH_2223_46,6/15/23,FEM,0.196027
602,DS_STARFISH_2223_46,6/15/23,KCHI,0.237888


In [31]:
All_ratio_df.to_excel('../AliceWhisperSync/StarFish2223_LexDivProportion_CHI_Agg.xlsx')

In [13]:
import pandas as pd
df = pd.read_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_TeacherQ.xlsx', index_col=0)
df_new = df.groupby('Subject ID')[['Total Sentences', 'Question Sentences', 'Non-Question Sentences', 'Question Responses', 'Non-Question Responses']].mean()
df_new.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_TeacherQ_Average.xlsx')
df = pd.read_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_KChildQ.xlsx', index_col=0)
df_new = df.groupby('Subject ID')[['Total Sentences', 'Question Sentences', 'Non-Question Sentences', 'Question Responses', 'Non-Question Responses']].mean()
df_new.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_KChildQ_Average.xlsx')

In [10]:
dic_tddd = pd.read_csv('../AliceWhisperSync/starfish2223PLS.csv')
# Step 1: Create the dictionary from the DataFrame
hearing_status_dict = dict(zip(dic_tddd['Subject'], dic_tddd['hearingStatus']))

df_get = pd.read_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_TeacherQ.xlsx')
# Step 2: Apply the dictionary to another DataFrame
# Assuming the other DataFrame is named 'df_other'
df_get['hearingStatus'] = df_get['Subject ID'].map(hearing_status_dict)
df_get.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_TeacherQ.xlsx', index_label=False)

df_get = pd.read_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_KChildQ.xlsx')
# Step 2: Apply the dictionary to another DataFrame
# Assuming the other DataFrame is named 'df_other'
df_get['hearingStatus'] = df_get['Subject ID'].map(hearing_status_dict)
df_get.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_KChildQ.xlsx', index_label=False)

In [6]:
import pandas as pd
from tqdm import tqdm

all_df = pd.read_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

all_responses_df = pd.DataFrame()
summary_list = []
processed_dfs = []
CHIList = ['KCHI']

response_time_window = 2.5
grouped = all_df.groupby(['subject ID', 'date'])
total_groups = len(grouped)

for i, ((recorder, date), group_df) in enumerate(tqdm(grouped, total=total_groups, desc="Processing")):
    response_df = pd.DataFrame(columns=['Question', 'Response', 'Response2Response', 'Question Time Start', 'Question Time End', 'Response Time Start', 'Response Time End', 'Response2Response Time Start', 'Response2Response Time End', 'Latencies_1', 'Latencies_2', 'Question Speaker', 'Response Speaker', 'Response2Response Speaker', 'File Name'])
    total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs, Q_RRs, NQ_RRs = 0, 0, 0, 0, 0, 0, 0

    num_rows = len(group_df) - 1

    # 遍历当前组的每一行
    index = -1
    for _, row in group_df.iterrows():
        index += 1
        if index == num_rows:
            break
        speaker = row['speaker_class']  # 假设speaker_class是Column O
        sentence = row['sentence']  # 假设sentence是Column E
        question_start = row['start_sec']
        question_end = row['end_sec']
        file_name = recorder + '_' + date

        total_Sen += 1

        if speaker in CHIList and '?' in str(sentence):
            Q_sen += 1
            index_2 = index + 1
            response_row = group_df.iloc[index_2]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])
                response_start = response_row['start_sec']
                response_end = response_row['end_sec']

                if response_speaker in CHIList and '?' in str(response_sen):
                    break

                if response_speaker in ['FEM'] and response_sen != 'nan':
                    Q_Rs += 1
                    group_df.at[index_2, 'Resp'] = 1
                    index_2R = index_2 + 1
                    response2response_row = group_df.iloc[index_2R]
                    while response2response_row['start_sec'] <= response_end + response_time_window and response2response_row['start_sec'] >= response_end:
                        #print('index_2R', index_2R)
                        response2response_speaker = response2response_row['speaker_class']
                        response2response_sen = str(response2response_row['sentence'])
                        response2response_start = response2response_row['start_sec']
                        response2response_end = response2response_row['end_sec']
                        
                        if response2response_speaker in CHIList and response2response_sen != 'nan':
                            Q_RRs += 1
                            new_row = pd.DataFrame({
                                'Question': [sentence],
                                'Response': [response_sen],
                                'Response2Response': [response2response_sen],
                                'Question Time Start': [question_start],
                                'Question Time End': [question_end],
                                'Response Time Start': [response_start],
                                'Response Time End': [response_end],
                                'Response2Response Time Start': [response2response_start],
                                'Response2Response Time End': [response2response_end],
                                'Latencies_1': [response_start - question_end],
                                'Latencies_2': [response2response_start - response_end],
                                'Question Speaker': [speaker],
                                'Response Speaker': [response_speaker],
                                'Response2Response Speaker': [response2response_speaker],
                                'File Name': [file_name]
                            })
                            response_df = pd.concat([response_df, new_row], ignore_index=True)
                            group_df.at[index_2R, 'Resp2Resp'] = 1
                            break
                            
                        index_2R += 1
                        if index_2R > num_rows:
                            break
                        response2response_row = group_df.iloc[index_2R]

                index_2 += 1
                if index_2 > num_rows:
                    break
                response_row = group_df.iloc[index_2]

        elif speaker in CHIList:
            NQ_sen += 1
            index_3 = index + 1
            response_row = group_df.iloc[index_3]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])
                response_start = response_row['start_sec']
                response_end = response_row['end_sec']

                if response_speaker in ['FEM']:
                    NQ_Rs += 1
                    group_df.at[index_3, 'Resp'] = 1
                    
                    index_3R = index_3 + 1
                    response2response_row = group_df.iloc[index_3R]
                    while response2response_row['start_sec'] <= response_end + response_time_window and response2response_row['start_sec'] >= response_end:
                        #print('index_3R', index_3R)
                        response2response_speaker = response2response_row['speaker_class']
                        response2response_sen = str(response2response_row['sentence'])
                        response2response_start = response2response_row['start_sec']
                        response2response_end = response2response_row['end_sec']
                        
                        if response2response_speaker in CHIList and response2response_sen != 'nan':
                            NQ_RRs += 1
                            group_df.at[index_3R, 'Resp2Resp'] = 1
                            break
                        index_3R += 1
                        if index_3R > num_rows:
                            break
                        response2response_row = group_df.iloc[index_3R]
                    
                index_3 += 1
                if index_3 > num_rows:
                    break
                response_row = group_df.iloc[index_3]

    summary_list.append((recorder, date, total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs, Q_RRs, NQ_RRs, file_name))
    all_responses_df = pd.concat([all_responses_df, response_df])
    processed_dfs.append(group_df[group_df['speaker_class'].isin(['FEM'])])

# 将summary_list转换为DataFrame以更好地查看和分析
summary_df = pd.DataFrame(summary_list, columns=['Subject ID', 'Date', 'Total All Sentences', 'Question Sentences', 'Non-Question Sentences', 'Question Responses', 'Non-Question Responses', 'Question Response2Responses', 'Non-Question Response2Responses', 'File Name'])

summary_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_KChildQ_3-Seq.xlsx')
all_responses_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Details_KCHI_FEM_KChildQ_3-Seq.xlsx')

Processing: 100%|██████████| 151/151 [02:27<00:00,  1.03it/s]


In [7]:
import pandas as pd
from tqdm import tqdm

all_df = pd.read_csv('../AliceWhisperSync/Cleaned_AggregatedData_CHI.csv')

all_responses_df = pd.DataFrame()
summary_list = []
processed_dfs = []
CHIList = ['KCHI']

response_time_window = 2.5
grouped = all_df.groupby(['subject ID', 'date'])
total_groups = len(grouped)

for i, ((recorder, date), group_df) in enumerate(tqdm(grouped, total=total_groups, desc="Processing")):
    response_df = pd.DataFrame(columns=['Question', 'Response', 'Response2Response', 'Question Time Start', 'Question Time End', 'Response Time Start', 'Response Time End', 'Response2Response Time Start', 'Response2Response Time End', 'Latencies_1', 'Latencies_2', 'Question Speaker', 'Response Speaker', 'Response2Response Speaker', 'File Name'])
    total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs, Q_RRs, NQ_RRs = 0, 0, 0, 0, 0, 0, 0

    num_rows = len(group_df) - 1

    # 遍历当前组的每一行
    index = -1
    for _, row in group_df.iterrows():
        index += 1
        if index == num_rows:
            break
        speaker = row['speaker_class']  # 假设speaker_class是Column O
        sentence = row['sentence']  # 假设sentence是Column E
        question_start = row['start_sec']
        question_end = row['end_sec']
        file_name = recorder + '_' + date

        total_Sen += 1

        if speaker in ['FEM'] and '?' in str(sentence):
            Q_sen += 1
            index_2 = index + 1
            response_row = group_df.iloc[index_2]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])
                response_start = response_row['start_sec']
                response_end = response_row['end_sec']

                if response_speaker in ['FEM'] and '?' in str(response_sen):
                    break

                if response_speaker in CHIList and response_sen != 'nan':
                    Q_Rs += 1
                    group_df.at[index_2, 'Resp'] = 1
                    index_2R = index_2 + 1
                    response2response_row = group_df.iloc[index_2R]
                    while response2response_row['start_sec'] <= response_end + response_time_window and response2response_row['start_sec'] >= response_end:
                        #print('index_2R', index_2R)
                        response2response_speaker = response2response_row['speaker_class']
                        response2response_sen = str(response2response_row['sentence'])
                        response2response_start = response2response_row['start_sec']
                        response2response_end = response2response_row['end_sec']
                        
                        if response2response_speaker in ['FEM'] and response2response_sen != 'nan':
                            Q_RRs += 1
                            new_row = pd.DataFrame({
                                'Question': [sentence],
                                'Response': [response_sen],
                                'Response2Response': [response2response_sen],
                                'Question Time Start': [question_start],
                                'Question Time End': [question_end],
                                'Response Time Start': [response_start],
                                'Response Time End': [response_end],
                                'Response2Response Time Start': [response2response_start],
                                'Response2Response Time End': [response2response_end],
                                'Latencies_1': [response_start - question_end],
                                'Latencies_2': [response2response_start - response_end],
                                'Question Speaker': [speaker],
                                'Response Speaker': [response_speaker],
                                'Response2Response Speaker': [response2response_speaker],
                                'File Name': [file_name]
                            })
                            response_df = pd.concat([response_df, new_row], ignore_index=True)
                            group_df.at[index_2R, 'Resp2Resp'] = 1
                            break
                            
                        index_2R += 1
                        if index_2R > num_rows:
                            break
                        response2response_row = group_df.iloc[index_2R]

                index_2 += 1
                if index_2 > num_rows:
                    break
                response_row = group_df.iloc[index_2]

        elif speaker in ['FEM']:
            NQ_sen += 1
            index_3 = index + 1
            response_row = group_df.iloc[index_3]
            while response_row['start_sec'] <= question_end + response_time_window and response_row['start_sec'] >= question_end:
                response_speaker = response_row['speaker_class']
                response_sen = str(response_row['sentence'])
                response_start = response_row['start_sec']
                response_end = response_row['end_sec']

                if response_speaker in CHIList:
                    NQ_Rs += 1
                    group_df.at[index_3, 'Resp'] = 1
                    
                    index_3R = index_3 + 1
                    response2response_row = group_df.iloc[index_3R]
                    while response2response_row['start_sec'] <= response_end + response_time_window and response2response_row['start_sec'] >= response_end:
                        #print('index_3R', index_3R)
                        response2response_speaker = response2response_row['speaker_class']
                        response2response_sen = str(response2response_row['sentence'])
                        response2response_start = response2response_row['start_sec']
                        response2response_end = response2response_row['end_sec']
                        
                        if response2response_speaker in ['FEM'] and response2response_sen != 'nan':
                            NQ_RRs += 1
                            group_df.at[index_3R, 'Resp2Resp'] = 1
                            break
                        index_3R += 1
                        if index_3R > num_rows:
                            break
                        response2response_row = group_df.iloc[index_3R]
                    
                index_3 += 1
                if index_3 > num_rows:
                    break
                response_row = group_df.iloc[index_3]

    summary_list.append((recorder, date, total_Sen, Q_sen, NQ_sen, Q_Rs, NQ_Rs, Q_RRs, NQ_RRs, file_name))
    all_responses_df = pd.concat([all_responses_df, response_df])
    processed_dfs.append(group_df[group_df['speaker_class'].isin(CHIList)])

# 将summary_list转换为DataFrame以更好地查看和分析
summary_df = pd.DataFrame(summary_list, columns=['Subject ID', 'Date', 'Total All Sentences', 'Question Sentences', 'Non-Question Sentences', 'Question Responses', 'Non-Question Responses', 'Question Response2Responses', 'Non-Question Response2Responses', 'File Name'])

summary_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Ana_KCHI_FEM_TeacherQ_3-Seq.xlsx')
all_responses_df.to_excel('../AliceWhisperSync/StarFish2223_QR_Details_KCHI_FEM_TeacherQ_3-Seq.xlsx')

Processing: 100%|██████████| 151/151 [03:07<00:00,  1.24s/it]
